Notebook to generate complete TurnSalmon data (including bounding boxes) from labeled track endpoints. 

In [1]:
# Code paths
import os
import sys
import yaml
import pandas as pd
import numpy as np

sys.path.append(os.path.split(os.getcwd())[0] + '\\helpers')
sys.path.append(os.path.split(os.getcwd())[0] + '\\evaluation\\py-motmetrics')
from motmetrics_evaluation import load_txt_data_for_eval
from comp_utils import get_salmon_ID_and_comp_type_from_comp_ID, get_comp_id_from_salmon_ID_and_comp_type

In [2]:
# Load data
gt_labels_path = 'C:\\Users\\espen\\Documents\\work\\PhD\\papers\\salmon_tracking\\data\\datasets\\without_boxes\\TS_val\\long_range_matches_korsneset_20240508_analysis1.xlsx'
gt_data_path = 'C:\\Users\\espen\\Documents\\work\\PhD\\papers\\salmon_tracking\\data\\datasets\\without_boxes\\TS_val\\not_track\\output\\salmon_tracking\\analysis1\\MOT_results.txt'

with open('\\'.join(gt_data_path.split('\\')[:-1]) + '\\config.yml', 'r') as f:
    config = yaml.safe_load(f)


In [3]:
# Format gt trackers
gt_data = load_txt_data_for_eval(gt_data_path)
gt_data[:,1] = gt_data[:,1].astype(float).astype(int).astype(str)
kb = gt_data[:,2:6].astype(float)
kb = np.vstack([kb[:,0]-kb[:,2]/2, kb[:,1]-kb[:,3]/2, kb[:,2], kb[:,3]]).T
gt_data[:,2:6] = kb.astype(str)
gt_data[:,1], _ = get_salmon_ID_and_comp_type_from_comp_ID(gt_data[:,1].astype(float).astype(int))
gt_labels =  np.array(pd.read_excel(gt_labels_path))

In [4]:
# Create labels on the form: comp_id, frame_num_start, xywh_start, frame_num_end, xywh_end, label_type (12 vals)

id = 0
prev_same_val = -1
labled_gt = []
finished_dets = []

for i in range(gt_labels.shape[0]):
    if gt_labels[i][3] != prev_same_val:
        id = id + 1
        prev_same_val = gt_labels[i][3]
    label_starts = gt_data[gt_data[:,1].astype(int)==int(gt_labels[i][0])]
    label_ends = gt_data[gt_data[:,1].astype(int)==int(gt_labels[i][1])]
    labels = []
    for label_start in label_starts:
        for label_end in label_ends:
            if label_start[10] == label_end[10]:
                if int(label_start[0])%10 != 0:
                    print('err')
                    print(label_start)
                    print(label_end)
                if int(label_end[0])%10 != 0:
                    print('err')
                    print(label_start)
                    print(label_end)
                compid = get_comp_id_from_salmon_ID_and_comp_type(id, config['components'].index(label_start[10]))
                label = [[str(compid)] + [label_start[0]] + list(label_start[2:6]) + [label_end[0]] + list(label_end[2:6]) + [gt_labels[i][2]] + [label_start[10]]]
                labled_gt.append(label)



In [ ]:
# Save complete TurnSalmon data
gt_finished = np.concatenate(labled_gt)
np.savetxt('\\'.join(gt_data_path.split('\\')[:-1]) + '\\MOT_results_labled.txt', gt_finished.astype(str), fmt='%s', delimiter=',')
print('GT succesfully created. Size of array: ' + str(gt_finished.shape))

GT succesfully created. Size of array: (1277, 13)
